I want you to help me to transform a dataframe to show process state evolution though time. The context is the following: the dataframe has 3 columns: 1. id_exp: it is the process id; 2.num_tramite: state id; 3.fecha: date of state change. The final dataframe should contain, for each date (and there must be values for every day between the first and last date of the dataframe), the number of processes on each state, as the final objective is to get an area plot showing the evolution of state change over time. Each process can be in one state at a time. So process starts in one state, stays some time in that state, then changes to other state, until it stops in a final state.  Please analyze the problem step by step, consider the best way to achieve it in an efficient way (computationally), and explain each step of the calculation. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_tramitesX = pd.read_parquet('./data/tramita/tramites_autoconsumo.parquet')

In [3]:
columns_estados = ['id_exp','num_tramite','fecha_tramite']
df_tramitesX_estados = df_tramitesX[columns_estados]

In [4]:
df_tramitesX_estados = df_tramitesX_estados.assign(fecha=df_tramitesX_estados['fecha_tramite'].dt.date)
df_tramitesX_estados.drop('fecha_tramite', axis = 1, inplace = True)

In [5]:
df_tramitesX_estados.head()

,id_exp,num_tramite,fecha
45875,1127895,0,2021-12-23
45876,1127895,1,2022-02-09
45877,1127895,10,2022-10-14
45878,1127895,30,2022-10-27
45879,1127895,42,2022-10-27


# Número de procesos por estado/día con date_range

In [6]:
df = df_tramitesX_estados

In [7]:
# Step 1: Sort DataFrame
df = df.sort_values(by=["id_exp", "fecha"])

In [8]:
# Step 2: Calculate End Date
df["end_date"] = df.groupby("id_exp")["fecha"].shift(-1)
# shift mueve una fila hacia arriba dentro de cada grupo
df.head()

,id_exp,num_tramite,fecha,end_date
3323569,1083865,0,2021-12-22,2022-02-04
3323570,1083865,1,2022-02-04,2022-04-19
3323571,1083865,10,2022-04-19,2022-05-04
3323572,1083865,30,2022-05-04,2022-06-08
3323573,1083865,42,2022-06-08,2022-08-10


,NUMTRAM_ORI,NSEC_ORI,DENOMINACION_ORIGEN,NUMTRAM_DES,NSEC_DES,DENOMINACION_DESTINO,NPROCADMI,_CELONIS_CHANGE_DATE
Index,,,,,,,,
0,279.0,1.0,Justificación final,156.0,1.0,Procedimiento de Reintegro (Reintegra),515.0,2024-08-27 02:01:32.580
1,279.0,1.0,Liquidacion Expediente (justificación),82.0,3.0,Pago Final - Resolucion de Liquidacion (SOLO J...,247.0,2024-08-27 02:01:32.580
2,280.0,1.0,Resolución de Liquidación,156.0,1.0,Procedimiento de reintegro,211.0,2024-08-27 02:01:32.580
3,281.0,1.0,Resolución de Renuncia Parcial,82.0,1.0,"Pago - Reconocimiento de Obligación (ADO,O)",1186.0,2024-08-27 02:01:32.580
4,281.0,1.0,Resolución de Revisión,82.0,1.0,"Pago - Reconocimiento de Obligación (ADO,O)",124.0,2024-08-27 02:01:32.580


In [19]:
# qué tramites son último estado
tramites_finales_num = df[df['end_date'].isnull()].groupby('num_tramite').size()
tramites_finales_num

num_tramite
1      15152
10       489
30         6
41        21
42       949
50      3860
51       403
52        35
70      3303
82        64
100      682
152       23
278        3
283        1
286        1
287      851
309        2
dtype: int64

In [20]:
type(tramites_finales_num)

pandas.core.series.Series

In [31]:
df_lista_tramitesfinales = pd.DataFrame(tramites_finales_num).reset_index()
df_lista_tramitesfinales.columns = ['num_tramite','count']
df_lista_tramitesfinales.head()

,num_tramite,count
0,1,15152
1,10,489
2,30,6
3,41,21
4,42,949


In [35]:
def_tramites = pd.read_parquet('./data/tramita/def_proc_tramite_unicos.parquet')
def_tramites_autoconsumo = def_tramites[def_tramites['cod_proc']==884]
def_tramites_autoconsumo

,cod_proc,num_tramite,denom
411,884.0,287.0,Seguimiento
1480,884.0,283.0,Resolución No procede pérdida total
1647,884.0,309.0,Lista de Reserva Provisional por Insuficiencia...
1673,884.0,283.0,Propuesta Resolución de Pérdida de Derecho al ...
1692,884.0,286.0,Presentación Solicitud Desistimiento
3001,884.0,50.0,Resolución de Concesión
3994,884.0,51.0,Resolución desestimatoria - Insuficiencia Credito
5086,884.0,52.0,Resolucion Desistimiento
5435,884.0,51.0,Resolución desestimatoria - Incumple Requisitos
5863,884.0,55.0,Resolución de inadmisión


In [37]:
df_lectura = pd.merge(df_lista_tramitesfinales, def_tramites_autoconsumo[['num_tramite','denom']],
                      left_on = 'num_tramite',
                      right_on = 'num_tramite',
                      how = 'left')
df_lectura

,num_tramite,count,denom
0,1,15152,Presentación de la solicitud
1,10,489,Requerimiento de documentación
2,30,6,Evaluación de solicitud
3,30,6,Evaluación de justificación tras alegaciones
4,30,6,Evaluación de justificación
5,41,21,Propuesta Resolución Desestimatoria - Incumple...
6,41,21,Propuesta Resolución Desestimatoria - Insufici...
7,42,949,Propuesta Resolución Estimatoria de Concesión
8,42,949,Propuesta Resolución Estimatoria Abono Pérdida...
9,50,3860,Resolución de Concesión


In [ ]:
# Step 3: Expand Dates
max_date = df['fecha'].max()
expanded_rows = []
for _, row in df.iterrows():
    # para cada estado de cada expediente se genera una línea por cada día en el rango de fechas
    if pd.notnull(row["end_date"]):
        date_range = pd.date_range(start=row["fecha"], end=row["end_date"] - pd.Timedelta(days=1))
    else:
        date_range = pd.date_range(start=row["fecha"], end=max_date)  #  end date
    for date in date_range:
        expanded_rows.append((date, row["num_tramite"]))
expanded_df = pd.DataFrame(expanded_rows, columns=["fecha", "num_tramite"])
expanded_df.head()
# The underscore (_) in that line is a convention in Python used as a throwaway variable - it indicates that we don't care about or won't use that particular value.
# In df.iterrows(), the function returns two values for each row:

# The index number of the row
# The actual row data

# So when you write for _, row in df.iterrows():, you're saying "I want to work with the row data, 
# but I don't care about the index number." The underscore is just a placeholder for that index value we're choosing to ignore.

In [ ]:
expanded_df[expanded_df['fecha']=='2021-12-22'].count()
# para una fecha debe salir el número de expedientes en cualquier estado ese día

In [ ]:
expanded_df[(expanded_df['fecha']=='2021-12-22') 
    & (expanded_df['num_tramite']==0)].count()
# al ser el primer día del dataframe, todos son estado 0, presentación de solicitud

In [ ]:
# Step 4: Aggregate by Date and State
aggregated_df = expanded_df.groupby(["fecha", "num_tramite"]).size().reset_index(name="count")
aggregated_df.head()

In [ ]:
# Step 5: Pivot for Plotting
pivot_df = aggregated_df.pivot(index="fecha", columns="num_tramite", values="count").fillna(0)
pivot_df.tail()

In [ ]:
pivot_df.info()

In [ ]:
pivot_df.index
#979 fechas, desde 2021-12-22 hasta 2024-08-26

In [ ]:
# Create a complete date range from min to max date
complete_dates = pd.date_range(start=pivot_df.index.min(), 
                             end=pivot_df.index.max(),
                             freq='D')

# Find missing dates by comparing with your index
missing_dates = complete_dates.difference(pivot_df.index)

# Check if there are any missing dates
if len(missing_dates) > 0:
    print(f"Found {len(missing_dates)} missing dates:")
    print(missing_dates)
else:
    print("No missing dates found")

In [ ]:
# Step 6: Resample to include all dates and fill missing values with 0
complete_pivot_df = pivot_df.resample('D').asfreq().fillna(0)

In [ ]:
complete_pivot_df.index

In [ ]:
complete_pivot_df.to_parquet('./data/tramita/tramites_autoconsumo_ts_estados.parquet')

In [ ]:
column_states = [0,100]
df_toplot = complete_pivot_df[column_states]

In [ ]:
# Step 7: Plot
df_toplot.plot(kind="area", stacked=True, figsize=(12, 6))
plt.title("Process State Evolution Over Time")
plt.xlabel("Date")
plt.ylabel("Number of Processes")
plt.legend(title="State ID")
plt.show()

# Análisis de estados


In [ ]:
df_def_procedimientos= pd.read_parquet('./data/tramita/def_procedimientos.parquet')

In [ ]:
df_def_procedimientos_autoconsumo = df_def_procedimientos[df_def_procedimientos['NPROCADMI']==884]
df_def_procedimientos_autoconsumo.head(20)

In [ ]:
df_def_procedimientos_autoconsumo[df_def_procedimientos_autoconsumo['NUMTRAM_ORI']==1]

In [ ]:
df_def_procedimientos_autoconsumo[df_def_procedimientos_autoconsumo['NUMTRAM_ORI']==0]

# Estados iniciales

In [ ]:
df_analisis = df_tramitesX_estados.sort_values(by=["id_exp", "fecha"])
# Step 2: Calculate End Date
df_analisis["end_date"] = df_analisis.groupby("id_exp")["fecha"].shift(-1)
df_analisis.head()

In [ ]:
df_analisis[df_analisis['id_exp']==1083865]

In [ ]:
df_analisis[df_analisis['end_date'].isnull()]
# cuantos tramites son el último estado, debería ser el mismo que el número de expedientes

In [ ]:
len(df_analisis['id_exp'].unique())

In [ ]:
# cuáles son los últimos estados
df_tramites_finales = df_analisis[df_analisis['end_date'].isnull()]
df_tramites_finales.groupby('num_tramite').size()

In [ ]:
df_tramites_finales_1 = df_tramites_finales[
    (df_tramites_finales['end_date'].isnull()) 
    & (df_tramites_finales['num_tramite']==1) ]
df_tramites_finales_1

In [ ]:
df_tramites_finales_1['num_tramite'].unique()

In [ ]:
df_tramites_finales_1


In [ ]:
df_tramitesX[df_tramitesX['id_exp']==1094633]